In [1]:
import os
import json
import wandb
import collections
import numpy as np
from glob import glob
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  if __name__ == '__main__':


In [2]:
LABEL_MAP = {
    "person": 11,
    "rider": 12,
    "car": 13,
    "truck": 14,
    "bus": 15,
    "train": 16,
    "motor": 17,
    "bike": 18,
    "traffic light": 6,
    "traffic sign": 7,
}

SEGMENTATION_LABEL_MAP = {
    0:  "road",
    1:  "sidewalk",
    2:  "building",
    3:  "wall",
    4:  "fence",
    5:  "pole",
    6:  "traffic light",
    7:  "traffic sign",
    8:  "vegetation",
    9:  "terrain",
    10: "sky",
    11: "person",
    12: "rider",
    13: "car",
    14: "truck",
    15: "bus",
    16: "train",
    17: "motor",
    18: "bike",
    255: "unknown"
}

IMAGE_DIR = "./data/bdd100k/images/100k/train"
SEGMENTATION_DIR = "./data/bdd100k/labels/sem_seg/masks"

In [3]:
train_annotation_file = "./data/bdd100k/labels/bdd100k_labels_images_train.json"
with open(train_annotation_file) as json_file:
    train_annotations = json.load(json_file)

In [4]:
train_segmentation_labels = glob(os.path.join(SEGMENTATION_DIR, "train", "*"))
val_segmentation_labels = glob(os.path.join(SEGMENTATION_DIR, "val", "*"))
len(train_segmentation_labels), len(val_segmentation_labels)

(7000, 1000)

In [5]:
common_det_seg_files_train, uncommon_det_seg_files_train = [], []
for annotation in tqdm(train_annotations):
    image_file = annotation["name"]
    if os.path.isfile(
        os.path.join(SEGMENTATION_DIR, "train", image_file[:-4] + ".png")
    ):
        common_det_seg_files_train.append(annotation)
    else:
        uncommon_det_seg_files_train.append(annotation)


len(common_det_seg_files_train), len(uncommon_det_seg_files_train)

  0%|          | 0/69863 [00:00<?, ?it/s]

(2972, 66891)

In [6]:
columns = [
    "File-ID",
    "Image-Bbox",
    "Num-Annotated-Objects",
    "Weather",
    "Scene",
    "Time-of-Day",
    "Has-Seg-Mask"
]
columns += [SEGMENTATION_LABEL_MAP[i] for i in range(19)] + ["unknown"]


object_sum = {
    "person": 0,
    "rider": 0,
    "car": 0,
    "truck": 0,
    "bus": 0,
    "train": 0,
    "motor": 0,
    "bike": 0,
    "traffic light": 0,
    "traffic sign": 0,
}

segmentation_pixel_sum = {}
for i in range(19):
    segmentation_pixel_sum[i] = 0
segmentation_pixel_sum[255] = 0

In [7]:
with wandb.init(project="bdd100k", entity="av-demo", job_type="visualize-dataset"):
    table = wandb.Table(columns=columns)
    for idx in tqdm(range(len(common_det_seg_files_train))):
        sample_image_file = os.path.join(IMAGE_DIR, common_det_seg_files_train[idx]["name"])
        sample_mask_file = os.path.join(
            "./data/bdd100k/labels/sem_seg/masks/train",
            common_det_seg_files_train[idx]["name"][:-4] + ".png"
        )
        bbox_data = []
        width, height = Image.open(sample_image_file).size
        for label in common_det_seg_files_train[idx]["labels"]:
            if "box2d" in label.keys():
                bbox_data.append({
                    "position": {
                        "minX": label["box2d"]["x1"] / width,
                        "maxX": label["box2d"]["x2"] / width,
                        "minY": label["box2d"]["y1"] / height,
                        "maxY": label["box2d"]["y2"] / height
                    },
                    "class_id" : LABEL_MAP[label["category"]],
                    "box_caption": label["category"],
                })
                object_sum[label["category"]] += 1
        image = np.array(Image.open(sample_image_file))
        mask = np.array(Image.open(sample_mask_file))
        frequency_dict = dict(collections.Counter(list(mask.flatten())))
        frequency_list = []
        for i in range(19):
            frequency_list.append(0 if i not in frequency_dict.keys() else frequency_dict[i])
            segmentation_pixel_sum[i] += 0 if i not in frequency_dict.keys() else frequency_dict[i]
        frequency_list.append(0 if 255 not in frequency_dict.keys() else frequency_dict[255])
        segmentation_pixel_sum[255] += 0 if i not in frequency_dict.keys() else frequency_dict[i]
        image = wandb.Image(
            image,
            boxes={
                "ground_truth": {
                    "box_data": bbox_data,
                    "class_labels": {v: k for k, v in LABEL_MAP.items()}
                }
            },
            masks={
                "ground_truth": {
                    "mask_data": mask,
                    "class_labels": SEGMENTATION_LABEL_MAP
                }
            }
        )
        table.add_data(
            common_det_seg_files_train[idx]["name"][:-4],
            image,
            len(bbox_data),
            common_det_seg_files_train[idx]["attributes"]["weather"],
            common_det_seg_files_train[idx]["attributes"]["scene"],
            common_det_seg_files_train[idx]["attributes"]["timeofday"],
            True,
            *frequency_list
        )
    
    wandb.log({"BDD100k-Dataset": table})
    
    data = [[label, val] for (label, val) in object_sum.items()]
    table = wandb.Table(data=data, columns = ["Detection-Classes", "Frequencies"])
    wandb.log({
        "Detection-Class Frequencies" : wandb.plot.bar(
            table, "Detection-Classes", "Frequencies", title="Detection-Class Frequencies"
        )
    })
    
    data = [[SEGMENTATION_LABEL_MAP[label], val] for (label, val) in segmentation_pixel_sum.items()]
    table = wandb.Table(data=data, columns = ["Segmentation-Classes", "Pixel-Counts"])
    wandb.log({
        "Segmentation-Class Frequencies" : wandb.plot.bar(
            table, "Segmentation-Classes", "Pixel-Counts", title="Segmentation-Class Frequencies"
        )
    })

wandb: Currently logged in as: geekyrakshit (use `wandb login --relogin` to force relogin)


  0%|          | 0/2972 [00:00<?, ?it/s]